### I am trying to automate weekly pos txn data records upload from interswitch and visa (NG and RoA) which has always been hectic and unneccessarily time consuming.
### challenge
##### I need to be able pick the excel file based on some specified criteria eg. file name like, date modified betwen a date range ect.

In [197]:
import pyodbc
import pandas as pd
import numpy as np
import os
#from datetime import date

In [198]:
os.listdir()

['.ipynb_checkpoints',
 'Financial Sample.xlsx',
 'financial_data_df.xlsx',
 'Untitled.ipynb']

In [191]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-LHG2ON9;'
                      'Database=WideWorldImporters;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [192]:
##Bring In Excel files
##SHEETNAME = NONE SO WE CAN BRING IN THE ALL SHEETS AT A GO
financial_date_df = pd.read_excel('Financial Sample.xlsx', sheet_name=None)
financial_date_df

{'Sheet1':        DEPOSIT      LOAN  TOTAL_INCOME       Date   Status
 0     32370.00   16185.0      16185.00 2014-01-01  ACTIVE 
 1     26420.00   13210.0      13210.00 2014-01-01  ACTIVE 
 2     32670.00   21780.0      10890.00 2014-06-01  ACTIVE 
 3     13320.00    8880.0       4440.00 2014-06-01  ACTIVE 
 4     37050.00   24700.0      12350.00 2014-06-01  ACTIVE 
 ..         ...       ...           ...        ...      ...
 394   13027.20    7080.0       5947.20 2014-06-01  ACTIVE 
 395   18721.08   14535.0       4186.08 2014-06-01  ACTIVE 
 396   25134.40   13660.0      11474.40 2014-06-01  ACTIVE 
 397  678960.00  615000.0      63960.00 2014-06-01  ACTIVE 
 398   27968.00   15200.0      12768.00 2014-11-01  ACTIVE 
 
 [399 rows x 5 columns],
 'Sheet3':       DEPOSIT    LOAN  TOTAL_INCOME       Date   Status
 0    14713.50    8655       6058.50 2019-11-24  DORMANT
 1     6273.00    4920       1353.00 2019-11-25  DORMANT
 2     4539.00    2670       1869.00 2019-11-26  DORMANT
 3   

In [195]:
## I found out the output is not a dataframe, hence you has no attribute head and shape below.
financial_date_df.head()

AttributeError: 'dict' object has no attribute 'head'

In [196]:
financial_date_df.shape()

AttributeError: 'dict' object has no attribute 'shape'

In [193]:
type(financial_date_df)

dict

In [145]:
for sheet in financial_date_df:
    print(f"{sheet} - {financial_date_df[sheet].shape}")

Sheet1 - (399, 5)
Sheet3 - (77, 5)
Sheet2 - (324, 5)


In [149]:
all_sheets = pd.concat(financial_date_df)
#Shape of Concatenated Sheets
all_sheets.shape

(800, 5)

In [176]:
new_data_min_date = pd.to_datetime(all_sheets['Date'].max())
today = pd.to_datetime(date.today())

print(new_data_min_date)
print(today)

2020-09-07 00:00:00
2020-10-03 00:00:00


In [177]:
#GET MAX DATE FROM THE STAGING TABLE
dest_table_max_date = pd.read_sql('SELECT max(Date) FROM Purchasing.FINANCIAL_DATA',conn)

In [185]:
#pd.to_datetime(dest_table_max_date)
dest_table_max_date = pd.to_datetime(dest_table_max_date.stack()).item()
dest_table_max_date

Timestamp('2020-09-07 00:00:00')

#### With the below iF statemennt  am trying to confirm that the max_date in destination table is less than the min date in the source data before I wrap it arount the insert statement.

In [186]:
#pd.datetime(dest_table_max_date)

if today > dest_table_max_date:
    print('continue')
else:
    print('confirm you have most recent POS data')


continue


In [108]:
all_sheets.head()

DEPOSIT     LOAN  TOTAL_INCOME       Date   Status
Sheet1 0  32370.0  16185.0       16185.0 2014-01-01  ACTIVE 
       1  26420.0  13210.0       13210.0 2014-01-01  ACTIVE 
       2  32670.0  21780.0       10890.0 2014-06-01  ACTIVE 
       3  13320.0   8880.0        4440.0 2014-06-01  ACTIVE 
       4  37050.0  24700.0       12350.0 2014-06-01  ACTIVE

In [117]:
all_sheets.tail()

DEPOSIT      LOAN  TOTAL_INCOME       Date   Status
Sheet2 319   36031.50   28260.0       7771.50 2019-11-19  DORMANT
       320   70443.75   79560.0      -9116.25 2019-11-20  DORMANT
       321  656370.00  643500.0      12870.00 2019-11-21  DORMANT
       322  259037.50  292560.0     -33522.50 2019-11-22  DORMANT
       323    9322.80    2742.0       6580.80 2019-11-23  DORMANT

In [118]:
all_sheets.columns

Index(['DEPOSIT', 'LOAN', 'TOTAL_INCOME', 'Date', 'Status'], dtype='object')

In [119]:
#Export the dataframe t excel
all_sheets.to_excel(r'C:\Users\Dinma\Desktop\pyhton_sql\financial_data_df.xlsx', sheet_name='uth_test_1', index = False)

In [114]:
#Drop Table
#cursor.execute(''' DROP TABLE Purchasing.FINANCIAL_DATA ''')  
#conn.commit()

In [116]:
#Insert Statement
for index, row in all_sheets.iterrows():
    cursor.execute("INSERT INTO Purchasing.FINANCIAL_DATA (DEPOSIT, LOAN, TOTAL_INCOME, Date, Status) 
                   values(?,?,?,?,?)", row. DEPOSIT, row.LOAN, row.TOTAL_INCOME, row.Date, row.Status)
conn.commit()

In [120]:
cursor.close()